### 加载并显示图像

In [1]:
import cv2

image = cv2.imread("jp.png")
# image是numpy数组
print(type(image))
# 高 x 宽 x 颜色通道数
(h, w, d) = image.shape
print("width={}, height={}, depth={}".format(w, h, d))

cv2.imshow("Image", image)
# 等待用户按键，delay表示等待毫秒数，传值0或不传值表示一直等待
cv2.waitKey()
cv2.destroyAllWindows()

<class 'numpy.ndarray'>
width=600, height=322, depth=3


### 访问单个像素

![RGB](opencv_tutorial_gradient_rgb.jpg)

首先，您可能会问：

> 什么是像素？

所有图像都由像素组成，这些像素是图像的原始构建块。图像由网格中的像素组成。640 x 480图像具有640列（宽度）和480行（高度）。有 640 * 480 = 307200 具有这些尺寸的图像中的像素。


灰度图像中的每个像素都有一个代表灰度的值。在OpenCV中，有256种灰度（从0到255）。因此，灰度图像将具有与每个像素关联的灰度值。

彩色图像中的像素具有其他信息。在学习图像处理时，您很快就会熟悉几种颜色空间。为简单起见，我们仅考虑RGB颜色空间。

在OpenCV彩色图像中，RGB（红色，绿色，蓝色）颜色空间中的每个像素都有一个三元组： （B，G，R ）。


请注意，顺序是BGR而不是RGB。这是因为多年前首次开发OpenCV时，标准是BGR订购。多年来，标准现已成为RGB，但OpenCV仍保持这种“传统” BGR顺序以确保不存在现有代码中断。

BGR 3元组中的每个值的范围为 [ 0 ，255 ] 。OpenCV中RGB图像中的每个像素有多少种颜色可能性？这很容易：256 * 256 * 256 = 16777216 。


既然我们确切地知道什么是像素，让我们看看如何检索图像中单个像素的值：

In [2]:
(B, G, R) = image[100, 50]
print("R={}, G={}, B={}".format(R, G, B))

R=45, G=53, B=38


### 数组切片和裁剪

提取“感兴趣区域”（ROI, regions of interest）是图像处理的一项重要技能。

现在，让我们*手动*提取ROI。这可以通过数组切片来实现。

In [3]:
roi = image[60:160, 320:420]
cv2.imshow("ROI", roi)
cv2.waitKey()
cv2.destroyAllWindows()

### 调整图像大小

In [4]:
# 忽略宽高比，将图片尺寸调整为200x200px
resized = cv2.resize(image, (200, 200))
cv2.imshow("Fixed Resizing", resized)
cv2.waitKey()
cv2.destroyAllWindows()

我们调整了忽略宽高比的图像的大小。显示图像已调整大小，但由于我们未考虑纵横比，因此现在已失真。
让我们计算原始图像的长宽比，并使用它来调整图像的大小，以使其看起来不会变形和变形：

In [5]:
# 固定大小和扭曲宽高比，调整宽度为300px，根据宽高比计算新的高度
r = 300.0 / w
dim = (300, int(h * r))
resized = cv2.resize(image, dim)
cv2.imshow("Aspect Ratio Resize", resized)
cv2.waitKey()
cv2.destroyAllWindows()

### 旋转影像

回忆一下中学/中学几何课中有关单位圆的信息，您将可以提醒自己，**正角是逆时针方向**，**负角是顺时针方向。**

以左上角为原点，水平x轴，竖直y轴，旋转矩阵M

旋转矩阵（以坐标原点为转轴逆时针旋转θ度）

(x, y) 旋转后坐标为 M[:, :2] * [[x], [y]] 

[[cosθ, sinθ],

[-sinθ, cosθ]]

旋转矩阵（以点(x', y')为转轴逆时针旋转θ度）

(x, y) 旋转后坐标为 M[:, :2] * [[x], [y]] + M[:, 2:]

[[cosθ, sinθ, x'(1 - cosθ) - y' sinθ],

[-sinθ, cosθ, x' sinθ + y'(1 - cosθ)]]

旋转矩阵（以点(x', y')为转轴逆时针旋转θ度，以点(x', y')为中心，以scale为比例放缩）

[[scale cosθ, scale sinθ, x'(1 - scale cosθ) - y' scale sinθ],

[-scale sinθ, scale cosθ, x' scale sinθ + y'(1 - scale cosθ)]]

In [6]:
center = (w // 2, h // 2)

# 以图片中心点为转轴，顺时针旋转45度，放缩比例为1.0，得到旋转矩阵
M = cv2.getRotationMatrix2D(center, -45, 1.0)

# 将原图片像素坐标按照旋转矩阵做仿射运算，得到旋转后的像素坐标，并指定返回图片显示范围 [0 ~ w, 0 ~ h]
rotated = cv2.warpAffine(image, M, (w, h))

cv2.imshow("OpenCV Rotation", rotated)
cv2.waitKey()
cv2.destroyAllWindows()

### 平滑处理

在许多图像处理管道中，我们必须对图像进行模糊处理以减少高频噪声，从而使我们的算法更容易检测和理解图像的实际*内容*，而不仅仅是 使“混淆”算法的*噪声*。在OpenCV中，对图像进行模糊处理非常容易，并且有多种方法可以完成图像处理。

In [7]:
# 将具有11x11内核的高斯模糊应用于图像以使其平滑，
# 在减少高频噪声时很有用
blurred = cv2.GaussianBlur(image, (11, 11), 0)
cv2.imshow("Blurred", blurred)
cv2.waitKey()
cv2.destroyAllWindows()

### 在图像上绘图

在本节中，我们将在输入图像上绘制矩形，圆形和直线。我们还将在图像上覆盖文本。

在继续使用OpenCV在图像上进行绘制之前，请注意，*在图像上进行绘制操作是就地执行的*。因此，在每个代码块的开头，我们制作原始图像的副本，并将副本存储为 output 。然后，我们继续绘制称为 output 就位，因此我们不会破坏原始图像。

绘制一个矩形：

In [8]:
# 将变量复制作为副本
output = image.copy()
# 绘制指定矩形，参数包含 矩形两个不相邻的顶点坐标，线条颜色，线条宽度
cv2.rectangle(output, (320, 60), (420, 160), (0, 0, 255), 2)
cv2.imshow("Rectangle", output)
cv2.waitKey()
cv2.destroyAllWindows()

绘制一个圆：

In [9]:
output = image.copy()
# 绘制指定圆，参数包含 圆心坐标，半径，线条颜色，线条宽度（-1表示实心）
cv2.circle(output, (300, 150), 20, (255, 0, 0), -1)
cv2.imshow("Circle", output)
cv2.waitKey()
cv2.destroyAllWindows()

绘制一条直线：

In [10]:
output = image.copy()
# 绘制指定直线，参数包含 直线上两个不同的点坐标，线条颜色，线条宽度
cv2.line(output, (60, 20), (400, 200), (0, 0, 255), 5)
cv2.imshow("Line", output)
cv2.waitKey()
cv2.destroyAllWindows()

绘制文本：

In [11]:
output = image.copy()
# 绘制文本，参数包含 文本内容，起点，字体，大小，颜色，粗细宽度
cv2.putText(output, "OpenCV + Jurassic Park!!!", (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
cv2.imshow("Text", output)
cv2.waitKey()
cv2.destroyAllWindows()